In [1]:
import rioxarray as rxr
import geopandas as gpd
import matplotlib.pyplot as plt
from loguru import logger
import pdb

from valleyfloor.process_topography import process_topography
from valleyfloor.delineate_reaches import delineate_reaches
from valleyfloor.utils import setup_wbt

logger.enable("valleyfloor")

In [2]:
wbt = setup_wbt("~/opt/WBT/", "../working_dir")

dem = rxr.open_rasterio("../data/input/dem.tif", masked=True).squeeze()
flowlines = gpd.read_file("../data/input/flowlines.shp")

In [3]:
dataset, aligned_flowlines = process_topography(dem, flowlines, wbt)

2024-10-08 22:07:11.545 | INFO     | valleyfloor.process_topography:process_topography:36 - process topography
/Users/arthurkoehl/Library/Caches/pypoetry/virtualenvs/slopes-mqMb1mKH-py3.10/lib/python3.10/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [4]:
from slopes.subbasins import label_subbasins
from slopes.hillslopes import label_hillslopes

In [5]:
subbasins = label_subbasins(dataset['flow_dir'], dataset['flow_acc'], dataset['flowpaths'], wbt)

In [ ]:
hillslopes = label_hillslopes(dataset['flowpaths'], dataset['flow_dir'], dataset['flow_acc'], subbasins)

1.0


/Users/arthurkoehl/Library/Caches/pypoetry/virtualenvs/slopes-mqMb1mKH-py3.10/lib/python3.10/site-packages/numba/typed/typeddict.py:34: NumbaTypeSafetyWarning: unsafe cast from int64 to uint8. Precision may be lost.
  d[key] = value


In [7]:
from slopes.utils import finite_unique
from valleyfloor.flow.hillslope import label_drainage_sides
import numpy as np

flow_paths = dataset['flowpaths']
flow_acc = dataset['flow_acc']
flow_dir = dataset['flow_dir']
assert(set(finite_unique(subbasins)) == set(finite_unique(flow_paths)))

hillslopes = flow_paths.copy()
hillslopes.data = np.zeros_like(flow_paths)

for streamID in finite_unique(subbasins):
    if streamID > 7:
        break
    print(streamID)
    basin_mask = (subbasins == streamID)
    flowpath_clipped = flow_paths.where(basin_mask)
    flowacc_clipped = flow_acc.where(basin_mask)
    flowdir_clipped = flow_dir.where(basin_mask)
    #basins = label_drainage_sides(flowpath_clipped, flowdir_clipped,
    #                                  flowacc_clipped)
    #mask = ~np.isnan(basins)
    #hillslopes = hillslopes.where(~mask, other=basins)

1.0
2.0
3.0
4.0
5.0
6.0
7.0


In [8]:
from valleyfloor.flow.graph import WBT_DIRMAP
from valleyfloor.flow.graph import flowdir_to_graph
from valleyfloor.flow.graph import trace_flowpaths

def _is_on_edge(row_ind, col_ind, array):
    nrow,ncol = array.shape
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            new_r = row_ind + dx
            new_c = col_ind + dy
            if new_r >= nrow or new_r < 0:
                return True
            if new_c >= ncol or new_c < 0:
                return True
            value = array[new_r, new_c]
            if not np.isfinite(value):
                return True

    return False

def _inds_of_inlet(flowpath, flowacc):
    condition = np.isfinite(flowpath)
    fa = flowacc.where(condition, drop=False)
    min_point = np.nanargmin(fa.data)
    min_x, min_y = np.unravel_index(min_point, fa.data.shape)
    return min_x.item(), min_y.item()

def _inds_of_outlet(flowpath, flowacc):
    # get inds of flowpath with max flow acc and min flowacc
    condition = np.isfinite(flowpath)
    fa = flowacc.where(condition, drop=False)
    max_point = np.nanargmax(fa.data)
    max_x, max_y = np.unravel_index(max_point, fa.data.shape)
    return max_x.item(), max_y.item()

In [9]:
flowpath = flowpath_clipped
flowacc = flowacc_clipped
flowdir = flowdir_clipped

In [15]:
inlet_r, inlet_c = _inds_of_inlet(flowpath, flowacc)
outlet_r, outlet_c = _inds_of_outlet(flowpath, flowacc)

basins = flowacc.copy()
basins.data = np.ones_like(flowacc.data)
basins.data[np.isfinite(flowpath)] = 0
basins.data[np.isnan(flowacc)] = np.nan
import skimage

In [16]:
# connected components
basins.data[np.isnan(basins.data)] = -9999
labels = skimage.measure.label(basins.data.astype(np.int32), connectivity=1)
basins.data = labels.astype(np.float64)
basins.data[np.isnan(flowacc)] = np.nan

In [ ]:
label_drainage_sides(flowpath, flowdir, flowacc)

/Users/arthurkoehl/Library/Caches/pypoetry/virtualenvs/slopes-mqMb1mKH-py3.10/lib/python3.10/site-packages/numba/typed/typeddict.py:34: NumbaTypeSafetyWarning: unsafe cast from int64 to uint8. Precision may be lost.
  d[key] = value
